In [ ]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('/content/drive/MyDrive/intents.json').read())

In [ ]:
words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


In [ ]:
print(documents)

[(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creators'], 'creator'

In [ ]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))
classes = sorted(set(classes))
print(len(classes))

38


In [ ]:

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))


In [ ]:

training = []
outputEmpty = [0] * len(classes)


In [ ]:

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)


In [ ]:
random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]

In [ ]:
print(len(training))

405


In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))


In [ ]:
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:

model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')
print('Done')

Epoch 1/200
81/81 [==============================] - 2s 6ms/step - loss: 3.5900 - accuracy: 0.0642
Epoch 2/200
81/81 [==============================] - 0s 4ms/step - loss: 3.3708 - accuracy: 0.1654
Epoch 3/200
81/81 [==============================] - 0s 5ms/step - loss: 3.1424 - accuracy: 0.1753
Epoch 4/200
81/81 [==============================] - 0s 6ms/step - loss: 2.8767 - accuracy: 0.2444
Epoch 5/200
81/81 [==============================] - 1s 8ms/step - loss: 2.5515 - accuracy: 0.3432
Epoch 6/200
81/81 [==============================] - 1s 7ms/step - loss: 2.3399 - accuracy: 0.3605
Epoch 7/200
81/81 [==============================] - 0s 5ms/step - loss: 2.1723 - accuracy: 0.4099
Epoch 8/200
81/81 [==============================] - 0s 3ms/step - loss: 1.9571 - accuracy: 0.4667
Epoch 9/200
81/81 [==============================] - 0s 6ms/step - loss: 1.7889 - accuracy: 0.5235
Epoch 10/200
81/81 [==============================] - 0s 5ms/step - loss: 1.5961 - accuracy: 0.5333
Epoch 11/

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize (sentence)
  sentence_words =[lemmatizer.lemmatize (word) for word in sentence_words]
  return sentence_words
def bag_of_words (sentence):
  sentence_words = clean_up_sentence (sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array (bag)

In [ ]:
def predict_class(sentence):
  bow = bag_of_words (sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'intent': classes [r[0]], 'probability': str(r[1])})
  return return_list
def get_response (intents_list, intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag'] == tag:
        result=random.choice(i['responses'])
        break
  return result



In [ ]:
  
print("GO! Bot is running!")
while True:
  message=input("")
  if message.lower() == "quit":
            break
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

GO! Bot is running!
how is weather today
1/1 [==============================] - 0s 24ms/step
Good to see you again!
good bye
1/1 [==============================] - 0s 25ms/step
Talk to you later
who are you
1/1 [==============================] - 0s 31ms/step
I am your helper
what you can help me with
1/1 [==============================] - 0s 22ms/step
You can ask me questions regarding college, and i will try to answer them
am i a good person
1/1 [==============================] - 0s 41ms/step
Sad to see you go :(
am i bad?
1/1 [==============================] - 0s 22ms/step
Come back soon
ok
1/1 [==============================] - 0s 38ms/step
I am glad I helped you
quit
